In [ ]:
# Importing necessary libraries

# Nearest neighbors will be the v1 model for sound drip
# will give logistic regressiona nd 
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from joblib import dump
from joblib import load
import pickle
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize

# textblob will be used to perform sentiment analysis down the road
# from textblob import TextBlob

# # Data is large and will need to be unzipped
# import zipfile

# # unzipping file
# with zipfile.ZipFile("./data_collection/final_data/DF_v1.pkl.zip","r") as zip_ref:
#     zip_ref.extractall(".")

# reading in pickled data
# song_list = pd.read_pickle("./DF_v1.pkl")

In [ ]:
# ALERT - THIS COULD POTENTIALLY BE USED TO PERFORM SENTIMENT ANALYSIS ON LYRICS DATA IN v2. FOR v1, DISREGARD

# # creating polarity and subjectivity lists to be populated

# lyrics_polarity = []
# lyrics_subjectivity = []

# # using a for loop to populate polarity and subjectivity lists
# for title in song_list_df["lyrics"]:
#     try:
#         blob_test = TextBlob = TextBlob(title)
#         blob_sentiment = blob_test.sentiment
#         # appending results to lists above
#         lyrics_polarity.append(blob_sentiment[0])
#         lyrics_subjectivity.append(blob_sentiment[1])
#     except:
#         lyrics_polarity.append(0)
#         lyrics_subjectivity.append(0)

# # adding polarity and subjectivity to df
# song_list['lyrics_polarity'] = lyrics_polarity
# song_list['lyrics_subjectivity'] = lyrics_subjectivity

# Loading Latest PKL

In [ ]:
# song_list = pickle.load("./DF_genres_hashed.pkl","rb")

X = pickle.load(open("./data/song_list_v4_hashed(1).pkl", "rb"))

In [ ]:
X["genres"].head()

In [ ]:
X.columns[0:20]

# Remove two NaN Values

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='63594c9b2f99411a8cbd18df04851fc4',client_secret='096168b2bd1f4378ae410726955c9ed8')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
def get_popularity(trackID):
    return sp.track(trackID)['popularity']

In [ ]:
get_popularity('5gNNUKhBzakQrbu8UaLori')

In [ ]:
X.loc[202786, 'popularity'] = 18

X.loc[202786]['popularity']

In [ ]:
get_popularity('25DO8ImBd90EhKEKuntxqN')

In [ ]:
X.loc[189355, 'popularity'] = 0

X.loc[189355]['popularity']

In [ ]:
X["genres_stripped"]

# Picking Final Updated DF

In [ ]:
X = X.drop(["availability"], axis=1)

In [ ]:
filename2 = './data/song_list_v5_hashed.pkl'
pickle.dump(X, open(filename2, 'wb'))

# Preparing for Scaling

In [ ]:
# dropping unnecessary columns
X = X.drop(["songid","artist","track", "duration_ms","genres","genres_stripped"], axis=1)

# checking that they're dropped
X.shape

In [ ]:
columns_for_scaling = X.columns[0:13]

In [ ]:
columns_genre_hashed = X.columns[13:]

In [ ]:
X[columns_for_scaling]

In [ ]:
X[columns_genre_hashed]

In [ ]:
# instantiating scaler
scaler = MinMaxScaler()

# scaling data
data_scaled = scaler.fit_transform(X[columns_for_scaling])

# confirming scaling took place
data_scaled[0]

In [ ]:
# pd.DataFrame(data_scaled,columns=X[columns_for_scaling].columns).hist(figsize=(8,8), normed=True);

In [ ]:
# data_for_model = np.concatenate((data_scaled,genres_hashed_array),axis=1)

In [73]:
len(data_scaled)

574018

# Normalization - Skipping for Now

In [74]:
import sys

In [75]:
np.set_printoptions(suppress=True)

In [76]:
data_scaled[7]

array([0.18556701, 0.85040161, 0.58375125, 0.0846    , 0.        ,
       1.        , 0.696     , 0.69854981, 1.        , 0.03660807,
       0.39780036, 0.8       , 0.396     ])

In [77]:
columns_for_normalization = columns_for_scaling

In [78]:
columns_for_normalization

Index(['popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [79]:
normalizer = Normalizer()
data_normalized = normalizer.fit_transform(data_scaled)

In [ ]:
# Valence isn't scaled appropriately
# will have to scale before normalizing
# pd.DataFrame(da,columns=columns_for_normalization).hist(figsize=(8,8), normed=True);

# K-Nearest Neighbors Classifier w/ Data scaled and Normalized

In [81]:
df = pickle.load(open("./data/song_list_v5_hashed.pkl", "rb"))

In [82]:
# instantiating model class
knn1 = NearestNeighbors(n_neighbors=10000,algorithm='kd_tree',n_jobs=-1)

# fitting model
model_1 = knn1.fit(data_normalized)

In [83]:
# running model to test output
knn_results1 = knn1.kneighbors([data_normalized[3]])[1]

In [84]:
knn_results1[0]

array([     3, 103114, 427498, ...,   4754, 238090, 225254])

In [85]:
pd.DataFrame(df.loc[1][0:19])

,1
songid,41RpZW2lxAdnqDd2nMBzLQ
popularity,15
acousticness,4.54e-05
artist,Hudson Mohawke
danceability,0.662
duration_ms,138960
energy,0.823
instrumentalness,0.952
key,4
liveness,0.343


In [86]:
dump(model_1, 'model3.joblib', compress=True)

['model3.joblib']

In [98]:
# converting song id column from df to an array

song_id_array3 = np.array(df['songid'])

filename5 = './data/song_id_array3.pkl'
pickle.dump(song_id_array3, open(filename5, 'wb'))

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
counter = 0
for result in knn_results1[0]:
    counter += 1
    print(counter)
    display(pd.DataFrame(df.loc[result][0:1]),pd.DataFrame(df.loc[result][1:2]),pd.DataFrame(df.loc[result][17:18]))

# Model w/ Scaling and no Normalization

In [ ]:
# instantiating model class
# knn2 = NearestNeighbors(n_neighbors=100,leaf_size=10,algorithm='kd_tree',n_jobs=-1)

# fitting model
# model2 = knn2.fit(data_scaled)

In [ ]:
# running model to test output
# knn_results2 = knn2.kneighbors([data_scaled[1]])[1]

In [ ]:
# knn_results2[0]

In [ ]:
# pd.DataFrame(df.loc[1][1:18])

In [ ]:
# counter = 0 
# for result in knn_results2[0]:
#     counter += 1 
#     print(counter)
#     display(pd.DataFrame(df.loc[result][0:1]),pd.DataFrame(df.loc[result][17:18]))

# Filtering KK For Shared Genres

In [ ]:
#loop takes KNN results and filters by source track genres
# old
filtered_list = []
source_song_index = knn_results1[0][0]
source_genre_list = df.loc[source_song_index]["genres_stripped"].split(",")
# print("source_genre_list created")
for source_genre in source_genre_list:
    source_genre = source_genre.strip(' ')
#     print("source_genre_created")
    for output_song_index in knn_results1[0][1:]:
        output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#         print("output_genre_list created")
        for output_genre in output_genre_list:
            output_genre = output_genre.strip(' ')
#             print("output_genre_created")
            if source_genre == output_genre:
                print(output_song_index,source_genre,output_genre)
                filtered_list.append(output_song_index)

In [ ]:
#loop takes KNN results and filters by source track genres
# new
filtered_lists = []
knn_result_ouputs = knn_results1[0][1:]
for source_song_index in range(0,5):
    filtered_list = []
    source_genre_list = genre_array[1][source_song_index]
    print(source_genre_list)
#     print("source_genre_list created")
#     for source_genre in source_genre_list:
#         source_genre = source_genre.strip(' ')
#     #     print("source_genre_created")
#         for output_song_index in knn_results1[0][1:]:
#             output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#     #         print("output_genre_list created")
#             for output_genre in output_genre_list:
#                 output_genre = output_genre.strip(' ')
#     #             print("output_genre_created")
#                 if source_genre == output_genre:
#                     print(output_song_index,source_genre,output_genre)
#                     filtered_list.append(output_song_index)

# knn_results1 = knn1.kneighbors([data_normalized[3]])[1]

In [ ]:
filtered_list.insert(0,3)

In [ ]:
filtered_list

In [107]:
filtered_lists = []
knn_result_ouputs = knn_results1[0][1:]
for source_song_index in range(0,5):
    filtered_list = []
    source_genre_list = genre_array[1][source_song_index]
    print(source_genre_list)
    # print("source_genre_list created")
#     for source_genre in source_genre_list:
#         source_genre = source_genre.strip(' ')
#     #     print("source_genre_created")
#         for output_song_index in knn_result_ouputs:
#             output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#     #         print("output_genre_list created")
#             for output_genre in output_genre_list:
#                 output_genre = output_genre.strip(' ')
#     #             print("output_genre_created")
#                 if source_genre == output_genre:
# #                     print(output_song_index,source_genre,output_genre)
# #                     filtered_list.append(output_song_index)
#     filtered_lists.append(filtered_list)

'bass music'
 'scottish electronic'
 'scottish hip hop'
 'uk experimental electronic'
 'wonky'


In [ ]:
X.shape

In [ ]:
genre_array = np.array([])

for row in range(0, 10000):
    genre = X.loc[row]['genres_stripped'].split(',')
    genre_array = np.append(genre_array, genre)

In [ ]:
genre_list = []

for row in range(0, 574018):
    genre = X.loc[row]['genres_stripped'].split(',')
    genre_list.append(genre)

In [ ]:
genre_array = np.array(genre_list)

In [ ]:
genre_array[0][0:10]

In [ ]:
nineteen_tracks = 0 
for filtered_list in filtered_lists:
    if len(filtered_list) > 19:
        nineteen_tracks += 1
print(nineteen_tracks / 100)

1000 = 38%
 X   = 150%

In [ ]:
150 / 38

In [ ]:
1000 * 4

In [ ]:
counter = 0
for index in filtered_list:
    counter += 1
    print(counter)
    display(pd.DataFrame(df.loc[index][0:1]),pd.DataFrame(df.loc[index][1:2]),pd.DataFrame(df.loc[index][17:18]))
    

In [ ]:
knn_results1[0][1:]

In [ ]:
df.loc[1]["genres_stripped"]

# Pickling model

In [ ]:
import pickle

filename = './models/knn_model_v1.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# pulling in the (now) pickled model
model_pickle = pickle.load(open(filename, 'rb'))

In [ ]:
# Loading model to test on scaled data (not pickled data)
pickle_results = model_pickle.kneighbors([data_scaled[1000]])[1]

# displaying output
pickle_results

# Pickling Scaled Data

In [ ]:
filename2 = './data_scaled.pkl'
pickle.dump(data_scaled, open(filename2, 'wb'))

# (OLD) Testing both pickles for dataset, model

In [ ]:
# importing libraries (even though they are visible above)
# to illustrate which packages are needed to create the function below

import pandas as pd 
from sklearn.neighbors import NearestNeighbors
import pickle


# this function predicts on a song id

def predict(id):
    #loads in pickled data and model
    X = pickle.load(open('./data_scaled.pkl', 'rb'))
    loaded_pickle = pickle.load(open('./models/knn_model_v1.pkl', 'rb'))
    #calculates results
    results = loaded_pickle.kneighbors([X[id]])[1]
    return results

In [ ]:
predict(0)

In [ ]:
content = {
    "audio_features": {
        "acousticness": 0.934,
        "danceability": 0.186,
        "energy": 0.107,
        "instrumentalness": 0,
        "key": 5,
        "liveness": 0.297,
        "loudness": -14.802,
        "mode": 1,
        "speechiness": 0.0347,
        "tempo": 107.095,
        "time_signature": 4,
        "valence": 0.149
    }
}

In [ ]:
dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')

In [ ]:
dataframe

In [ ]:
dataframe_scaled = scaler.transform(dataframe)

dataframe_scaled

In [ ]:
import pickle

filename = './models/scaler.pkl'
pickle.dump(scaler, open(filename, 'wb'))

In [ ]:
target = 'songid'

song_id_array = song_list[target]

In [ ]:
song_id_array

In [ ]:
sys.getsizeof(song_id_array)

In [ ]:
song_id_list = song_id_array.tolist()

In [ ]:
sys.getsizeof(song_id_list)

In [ ]:
import pickle

filename = 'Flask_API/SOUNDDRIP/data/song_id_array.pkl'
pickle.dump(song_id_array, open(filename, 'wb'))

In [ ]:
import pickle

filename = 'song_id_list2.pkl'
pickle.dump(song_id_list, open(filename, 'wb'))

In [ ]:
content = {
    "audio_features": {
        "acousticness": 0.934,
        "danceability": 0.186,
        "energy": 0.107,
        "instrumentalness": 0,
        "key": 5,
        "liveness": 0.297,
        "loudness": -14.802,
        "mode": 1,
        "speechiness": 0.0347,
        "tempo": 107.095,
        "time_signature": 4,
        "valence": 0.149
    }
}

def predict(content):
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scaler = pickle.load(open('models/scaler.pkl', 'rb'))
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scaler.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = pickle.load(open('./models/knn_model_v1.pkl', 'rb'))
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')

    return results


In [ ]:
predict(content)

In [ ]:
  # For-loop for returning 20 songs
def all_similarities(data_result):
    similar_songs = []
    print('song_id_list loading...')
    song_id_list = pickle.load(open('Flask_API/SOUNDDRIP/data/song_id_list.pkl', 'rb'))
    print('song_id_list loaded')
    print('beginning for loop...')
    
    for song_row in data_result[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict

In [ ]:
all_similarities(predict(content))

In [ ]:
sorted_list = sorted(similarities, key=lambda i: i['similarity'], 
                                                         reverse=True)[:20]
      json_dict = {"songs": sorted_list}
      #data = json.dumps(json_dict)
      return jsonify(json_dict), print('yay')

# Get Spotify API Token / Output Audio Features

In [ ]:
import spotipy
import spotipy.util as util

In [ ]:
USERNAME = 'augt12ocf9csxa4s8kbq9reg8' #your spotify username
CLIENT_ID = 'b954b92b9a674e6a9cf7322649da82d1' #set at your developer account
CLIENT_SECRET = '920cd41023994b1a99210243ce8967b2' #set at your developer account
REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

In [ ]:
token

In [ ]:
token_content = {'token': token}

In [ ]:
type(token_content)

In [ ]:
token_content['token']

In [ ]:
def get_id(token):
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    song_id = results['items'][0]['track']['id']
    return song_id

In [ ]:
def get_features(song_id):
    results_dict = sp.audio_features(song_id)[0]
    audio_features = {"audio_features":{key: results_dict[key] for key in results_dict.keys() & {'danceability', 'energy',
                                                                                                'key', 'loudness', 'mode',
                                                                                                'speechiness', 'acousticness',
                                                                                                'instrumentalness', 'liveness',
                                                                                                'valence', 'tempo', 'time_signature'}}}
        
    return audio_features

In [ ]:
 acoustical_features = get_features(get_id(token))

In [ ]:
acoustical_features

# Flask Predict function

In [ ]:
def predict(content):
    similar_songs = []
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scalar = load('scalar2.joblib')
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    dataframe_scaled = scalar.transform(dataframe)
    print('Dataframe scaled')
    print('Loading pickled model...')
    model = load('model2.joblib')
    print('Model loaded')
    results = model.kneighbors([dataframe_scaled][0])[1]
    print('Prediction executed')
    print('song_id_list loading...')
    song_id_list = load('song_id_list2.joblib')
    print('song_id_list loaded')
    
    print('beginning for loop...')
    for song_row in results[0][1:]:
        song_id = song_id_list[song_row]
        similar_songs.append({'similarity': [.99], 'values': song_id})
    json_dict = {"songs": similar_songs}
    return json_dict


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from flask import request
from joblib import dump
from joblib import load

import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify

In [ ]:
%timeit

predict(acoustical_features)

# Dump

In [ ]:
dump(model, 'model2.joblib', compress=True)

In [ ]:
# %time

# pickle_loaded = pickle.load(open('Flask_API/SOUNDDRIP/models/knn_model_v1.pkl', 'rb'))
# joblib_loaded = load('model.joblib')



In [ ]:
# dump(data_scaled, 'data_scaled.joblib', compress=True)

In [ ]:
dump(scaler, 'scalar2.joblib', compress=True)

In [ ]:
dump(song_id_list, 'song_id_list2.joblib', compress=True)